# Split large file into multiple

In [1]:
## testing multidark
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
import h5py
import astropy.io.fits as fits
import csv
import pandas as pd
import h5py
import tables
import pickle
import os
from astropy.table import Table
from astropy.coordinates import SkyCoord
from tqdm import tqdm
from astropy.io import ascii
import os

import warnings
warnings.filterwarnings("ignore")

import timeit

In [2]:
datadir = '/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p49/'

In [3]:
filename = 'hlist_0.67120.list'

In [4]:
count = 0
with open(datadir+filename) as fp:
    for line in fp:
        count += 1
        if count %5e6==0: 
            print(count)
            speech = script.read().splitlines()

print('number of non-blank lines', count)

5000000


NameError: name 'script' is not defined

## Splitting the files by line

In [4]:
NUM_OF_LINES=1e7
filename = 'hlist_0.67120.list'
with open(datadir+filename) as fin:
    fout = open(datadir+"mdpl2_rockstar_halos_z0p00_0.list","w")
    for i,line in enumerate(fin):
        fout.write(line)
        if (i+1)%NUM_OF_LINES == 0:
            fout.close()
            fout = open(datadir+"mdpl2_rockstar_halos_z0p49_%d.list"%(i/NUM_OF_LINES+1),"w")
            print("Outputting file")

    fout.close()

Outputting file
Outputting file
Outputting file
Outputting file
Outputting file
Outputting file
Outputting file
Outputting file
Outputting file
Outputting file
Outputting file
Outputting file
Outputting file


In [ ]:
import glob

datadir = '/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p00/'
files = glob.glob(datadir + 'mdpl2_rockstar_halos_z0p00_*.list')

for f in files:
    os.rename(datadir+f,'to.another.extension')

## Checking if files are readable

In [20]:
##Path to Scratch space

#outputdir = datadir+ 'z0p00/'

import glob
files = glob.glob(datadir + 'mdpl2_rockstar_halos_z0p25_*.txt')

col_dict = {0:'scale', 1:'id',  5:'pid', 10:'mvir', 11:'rvir', 12: 'rs', \
            17:'x', 18:'y', 19:'z', \
            38: 'Mmvir_all', 39:'M200b', 40:'M200c', 41:'M500c', \
            56: 'T/|U|', 63: 'Halfmass_Scale', 64: 'Acc_Rate_Inst', 65: 'Acc_Rate_100Myr', 66: 'Acc_Rate_1*Tdyn', 67: 'Acc_Rate_2*Tdyn', 68: 'Acc_Rate_Mpeak'}

cluster_df = pd.DataFrame(columns = list(col_dict.values()))

In [22]:
mvir_cut = 1e14

for f in files:
    #Reading the file as a pandas dataframe
    stock =os.path.basename(f)
    print(f)
    halo_df = pd.read_table(datadir+stock, comment='#', sep='\s+', encoding='utf-8', names=list(col_dict.values()), usecols=list(col_dict.keys()))
    
    #Masking
    mask = ~(halo_df[['mvir','M200b', 'M200c', 'M500c']] == 0).any(axis=1) ##Mass non-zero
    mask &= (halo_df.Mmvir_all > mvir_cut)
    mask &= (halo_df.pid == -1)

    halo_df = halo_df[mask]

    #Appending onto the cluster dataframe
    cluster_df = cluster_df.append(halo_df)
    print("Number of clusters appended: {:d}".format(len(halo_df)) ) 

/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p25/mdpl2_rockstar_halos_z0p25_13.txt
Number of clusters appended: 132
/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p25/mdpl2_rockstar_halos_z0p25_4.txt
Number of clusters appended: 1484
/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p25/mdpl2_rockstar_halos_z0p25_2.txt
Number of clusters appended: 1485
/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p25/mdpl2_rockstar_halos_z0p25_9.txt
Number of clusters appended: 1469
/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p25/mdpl2_rockstar_halos_z0p25_1.txt
Number of clusters appended: 1466
/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p25/mdpl2_rockstar_halos_z0p25_7.txt
Number of clusters appended: 1468
/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/z0p25/mdpl2_rockstar_halos_z0p25_10.txt
Number of clusters appended: 1388
/global/cscratch1/sd/zzhang13/MultiDark/MDPL2_ROCKSTAR_Halos/

In [11]:
halo_df

,scale,id,pid,mvir,rvir,rs,x,y,z,Mmvir_all,M200b,M200c,M500c,T/|U|,Halfmass_Scale,Acc_Rate_Inst,Acc_Rate_100Myr,Acc_Rate_1*Tdyn,Acc_Rate_2*Tdyn,Acc_Rate_Mpeak
62349,0.8013,11190745795,-1,2.568300e+14,1396.104980,189.520004,977.63849,984.39484,268.27951,2.568500e+14,2.760800e+14,2.241300e+14,1.630200e+14,0.6351,0.4406,20540.00,20540.00,12700.0,22890.0,14150.0
62427,0.8013,11190677404,-1,2.959400e+14,1463.655029,264.216003,970.30450,984.50977,216.39291,2.962600e+14,3.117000e+14,2.629100e+14,1.989100e+14,0.6787,0.5622,16540.00,16540.00,47200.0,30810.0,45900.0
62490,0.8013,11190186169,-1,1.934200e+14,1270.197021,159.910004,994.78571,918.02167,293.87903,1.934500e+14,2.050300e+14,1.744200e+14,1.374800e+14,0.6590,0.4406,21250.00,21250.00,26380.0,17500.0,10550.0
62560,0.8013,11190712060,-1,1.382100e+14,1135.583984,210.003006,962.81335,958.33453,228.51646,1.383900e+14,1.479700e+14,1.180900e+14,8.466600e+13,0.6960,0.6008,12860.00,12860.00,11870.0,21900.0,24490.0
62602,0.8013,11190677576,-1,2.484800e+14,1380.805054,301.346008,964.25586,953.05151,219.25894,2.484900e+14,2.633000e+14,2.127800e+14,1.436300e+14,0.6329,0.5498,11070.00,11070.00,13020.0,28180.0,30860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905015,0.8013,11182109457,-1,1.234000e+14,1093.480957,281.311005,903.01520,938.40613,950.33240,1.237400e+14,1.317000e+14,1.042000e+14,6.527300e+13,0.6813,0.6142,3750.00,3750.00,12220.0,23810.0,23600.0
905073,0.8013,11182075665,-1,1.221400e+14,1089.728027,188.899994,925.60718,990.00751,923.40710,1.221800e+14,1.285900e+14,1.070800e+14,7.438400e+13,0.6369,0.5378,5501.00,5501.00,9301.0,12520.0,13920.0
905101,0.8013,11182076287,-1,1.101300e+14,1052.770020,230.975998,917.62360,987.14172,906.52606,1.102500e+14,1.169700e+14,9.595000e+13,6.589300e+13,0.6763,0.5876,8751.00,8751.00,15170.0,13440.0,15810.0
905162,0.8013,11191096904,-1,1.007000e+14,1021.835022,222.481995,992.31769,938.09491,921.75800,1.007000e+14,1.074800e+14,8.593500e+13,5.664900e+13,0.6423,0.5498,71.44,71.44,3893.0,9928.0,12630.0
